# Shape segmentation

The notebooks in this folder replicate the experiments as performed for [CNNs on Surfaces using Rotation-Equivariant Features](https://doi.org/10.1145/3386569.3392437).

The current notebook replicates the shape segmentation experiments from section `5.2 Comparisons`.

## Imports
We start by importing dependencies.

In [1]:
# File reading and progressbar
import os.path as osp
import progressbar

import numpy as np
import random
import time

# PyTorch and PyTorch Geometric dependencies
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch_geometric.transforms as T
from torch_geometric.data import DataLoader
from torch_geometric.nn.inits import zeros
from torch import autograd
import os
#os.environ['CUDA_LAUNCH_BLOCKING'] = '1'



# Harmonic Surface Networks components
# Layers
from nn import (ECResNetBlock, ECMLP, LiftBlock, ExtConvCplx, ExtConvRe, SignalNonLin,
                ParallelTransportPool, ParallelTransportUnpool, FourierMLP, InvariantMLP,
                TangentLin, TangentNonLin, VectorDropout, ECHOBlock, TangentMeanPool)
# Utility functions
from utils.harmonic import magnitudes, norm2D
# Rotated MNIST dataset
from datasets import ShapeSeg
# Transforms
from transforms import (HarmonicPrecomp, VectorHeat, MultiscaleRadiusGraph, PoolPrecomp,
                        ScaleMask, FilterNeighbours, NormalizeArea, NormalizeAxes, Subsample)

## Settings
Next, we set a few parameters for our network. You can change these settings to experiment with different configurations of the network. Right now, the settings are set to the ones used in the paper.

In [2]:
# Band-limit for extended convolution
band_limit = 2

# Number of rings in the radial profile
n_corr_rings = 6

# Number of conv rings
n_conv_rings = 6

# Learn radial offset for correlations
offset = True;

# Number of filters per block
nf = [32, 64];

n_des = 16;
n_bins = 3;

# Ratios used for pooling
ratios=[1, 0.25]

# Radius of convolution for each scale
rad = 0.2;
radii = [rad, np.sqrt(2)*rad]

# Number of datasets per batch
batch_size = 1

# Number of classes for segmentation
n_classes = 8

## Dataset
To get our dataset ready for training, we need to perform the following steps:
1. Provide a path to load and store the dataset.
2. Define transformations to be performed on the dataset:
    - A transformation that computes a multi-scale radius graph and precomputes the logarithmic map.
    - A transformation that masks the edges and vertices per scale and precomputes convolution components.
3. Assign and load the datasets.

In [3]:
# 1. Provide a path to load and store the dataset.
# Make sure that you have created a folder 'data' somewhere
# and that you have downloaded and moved the raw datasets there
path = osp.join('data', 'ShapeSeg')

# 2. Define transformations to be performed on the dataset:
# Transformation that computes a multi-scale radius graph and precomputes the logarithmic map.
pre_transform = T.Compose((
    NormalizeArea(),
    MultiscaleRadiusGraph(ratios, radii, loop=True, flow='target_to_source', sample_n=1024),
    VectorHeat(max_lvl=len(ratios)-1),
    Subsample(),
    NormalizeAxes()
))
# Apply a random scale and random rotation to each shape
transform = T.Compose((
    T.RandomScale((0.85, 1.15)),
    T.RandomRotate(45, axis=0),
    T.RandomRotate(45, axis=1),
    T.RandomRotate(45, axis=2))
)

# Transformations that masks the edges and vertices per scale and precomputes convolution components.
scale0_transform = T.Compose((
    ScaleMask(0),
    FilterNeighbours(radii[0]),
    HarmonicPrecomp(n_conv_rings, n_corr_rings, band_limit, max_r=radii[0]))
)
scale1_transform = T.Compose((
    ScaleMask(0),
    FilterNeighbours(0.25*radii[0]),
    HarmonicPrecomp(n_conv_rings, n_corr_rings, band_limit, max_r=0.25*radii[0]))
)


# 3. Assign and load the datasets.
test_dataset = ShapeSeg(path, False, pre_transform=pre_transform)
test_loader = DataLoader(test_dataset, batch_size=batch_size)
train_dataset = ShapeSeg(path, True, pre_transform=pre_transform, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

## Network architecture
Now, we create the network architecture by creating a new `nn.Module`, `Net`. We first setup each layer in the `__init__` method of the `Net` class and define the steps to perform for each batch in the `forward` method. The following figure shows a schematic of the architecture we will be implementing:

<img src="img/resnet_architecture.png" width="800px" />

Let's get started!

In [4]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.lift = LiftBlock(3, nf[0], n_corr_rings, offset)
        
        self.resnet1 = ECResNetBlock(nf[0], nf[0], band_limit, n_conv_rings)
                
        self.resnet2 = ECResNetBlock(nf[0], nf[0], band_limit, n_conv_rings)
        
        self.resnet3 = ECResNetBlock(nf[0], nf[0], band_limit, n_conv_rings)

        self.echo = ECHOBlock(nf[0], n_des, n_bins, n_classes, band_limit, n_conv_rings)
        
    def forward(self, data):
        
        ###############
        ### Level 1 ###
        ###############
        
        # Select only the edges and precomputed components of the first scale
        data_scale0 = scale0_transform(data)
        data_scale1 = scale1_transform(data)
        
        attr_grad = (data_scale0.edge_index, data_scale0.pcmp_gather)

        attr_conv = (data_scale0.edge_index, data_scale0.pcmp_scatter, 
                      data_scale0.connection)   
                
        
        attr_echo = (data_scale0.edge_index, data_scale0.pcmp_scatter, data_scale0.pcmp_echo, data_scale0.connection)

                
        
        x = data.pos
        
        x = self.lift(x, *attr_grad)
                
        x = self.resnet1(x, *attr_conv)
        
        x = self.resnet2(x, *attr_conv)
        
        x = self.resnet3(x, *attr_conv)

        x = self.echo(x, *attr_echo)


        return F.log_softmax(x , dim=1)

        
        
        
                   


## Training

Phew, we're through the hard part. Now, let's get to training. First, move the network to the GPU and setup an optimizer.

In [5]:
# We want to train on a GPU. It'll take a long time on a CPU
device = torch.device('cuda')
# Move the network to the GPU
model = Net().to(device)
# Set up the ADAM optimizer with learning rate of 0.0076 (as used in H-Nets)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

Next, define a training and test function.

In [6]:
def test():
    # Set model to 'evaluation' mode
    model.eval()
    correct = 0
    total_num = 0
    for i, data in enumerate(test_loader):
        pred = model(data.to(device)).max(1)[1]
        correct += pred.eq(data.y).sum().item()
        total_num += data.y.size(0)
        
    stateFile =  '/home/tommy/Dropbox/specialMath/Harmonic/ECHONet/V7/plot/ShapeSeg/VFC_state_{}'
    
    acc = correct / total_num
    if acc > 0.925:
        torch.save(model.state_dict(), stateFile.format(acc))        

    return acc

def train(epoch, acc=0):
    # Set model to 'train' mode
    model.train()

    
    if epoch < 0:
        rate = 0.01
    else:
        rate = 0.01
        
    #if (acc > 0.91):
        #rate = 0.0025
        
    #if (epoch > 15 or acc > 0.91):
        #rate = 0.002
    if (epoch > 30 or acc > 92):
        rate = 0.001
    

    
   # if (epoch > 20 or acc > 0.92):
      #  rate = 0.0005
    
    for param_group in optimizer.param_groups:
        param_group['lr'] = rate

            
            
    # Sort out progress bar
    n_data = train_loader.__len__()
    widgets = [progressbar.Percentage(), progressbar.Bar(), 
              progressbar.AdaptiveETA(), ' | Loss:', progressbar.Variable('loss'),]

    bar = progressbar.ProgressBar(max_value=n_data, widgets=widgets)

    i = 0;
    totalL = 0.0;
    for data in train_loader:
    # Move training data to the GPU and optimize parameters
        #with autograd.detect_anomaly():
        optimizer.zero_grad()
        L = F.nll_loss(model(data.to(device)), data.y);
        totalL += L.item()
        i = i + 1
        bar.update(i, loss = (totalL / i) )
        L.backward()
        optimizer.step()
        
        '''
        if epoch > 0 && i % 120 == 0:
            test_acc = test()
            print("Data {} - Test: {:06.4f}".format(i, test_acc));
            model.train()
        '''

        
        


Train for 50 epochs.

In [7]:
print('Start training, may take a while...')
# Try with fewer epochs if you're in a timecrunch
acc = 0;
loadPath = '/home/tommy/Dropbox/specialMath/Harmonic/ECHONet/V7/plot/ShapeSeg/VFC_state_0.925'
model.load_state_dict(torch.load(loadPath))
for epoch in range(50):
    #train(epoch, acc)
    acc = test()
    #test_acc = test() 
    #acc = test_acc if test_acc > acc else acc
    print("Epoch {} - Test: {:06.4f}".format(epoch, acc))

Start training, may take a while...
Epoch 0 - Test: 0.9249


KeyboardInterrupt: 